In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./source_data/1976-2020-president.csv')

In [4]:
data = df[['year', 'state_po', 'candidate', 'candidatevotes', 'party_simplified']]
data = data.rename(columns = {'state_po': 'state', 'candidatevotes': 'votes', 'party_simplified': 'party'})
data = data.dropna()

data = data.groupby(['year', 'state', 'candidate'])[['votes', 'party']]\
        .apply(lambda table: pd.Series({'votes' : table['votes'].sum(), 
                              'party': table.sort_values(by='votes', ascending = False)['party'].iloc[0]}))\
        .reset_index()

data = data[data['candidate'] != 'PEROT, ROSS']
data = data.groupby(['year', 'state'])[['candidate', 'party', 'votes']]\
                .apply(lambda table : table.sort_values(by = 'votes', ascending = False)[:2])\
                .reset_index()\
                .drop(columns = 'level_2')

data = data.drop(columns = 'candidate')
data = data.pivot(index = ['year', 'state'], columns = 'party', values = 'votes').reset_index()

In [6]:
data['total'] = data['DEMOCRAT'] + data['REPUBLICAN']
data['dem_pct'] = data['DEMOCRAT']/data['total']
data['rep_pct'] = data['REPUBLICAN']/data['total']
data = data[['year', 'state', 'dem_pct', 'rep_pct']]
data.to_csv('./processed_data/1976-2020-president-two-party.csv', header = True, index = False)